In [1]:
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
from keras import backend
from keras.layers import Dense, Input, Lambda, LSTM, TimeDistributed
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model
from gensim.models.keyedvectors import KeyedVectors
import json
import nltk

c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.
c:\users\dh kim\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected 

In [2]:
def isNumber(s):
  try:
    float(s)
    return True
  except ValueError:
    return False

In [3]:
def gram(str):
    retlist=[]
    for i in range(1,4):
        for j in range(len(str)-i+1):
            retlist.append(str[j:j+i])
    return list(retlist)

print(gram(u"가나다라"))
def isNumber(s):
  try:
    float(s)
    return True
  except ValueError:
    return False


['가', '나', '다', '라', '가나', '나다', '다라', '가나다', '나다라']


In [63]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
y_name_id_dict = joblib.load("y_name_id_dict.dat")
y_id_list = [y_name_id_dict[x] for x in y_text_list]

In [64]:
from sklearn.model_selection import train_test_split
import re
vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(map(lambda i : i[1],x_text_list))
y_list = [y_name_id_dict[x] for x in y_text_list]
#print(x_text_list)
tmp_list = list(map(lambda i : i[1],x_text_list))
#print(x_text_list)
print(tmp_list[0])

#MD추천정품# 심플베드 엘레강스 고급형 접이식침대 더블사이즈115cm/수동/등받이6단으로 원하는각도조절


In [65]:
ngram_data=[]
for ls in tmp_list:
    data = re.findall("[A-Za-z0-9]+", ls)
    tmp=""
    for i in range(0,len(data)):
        if(isNumber(data[i]) is False):
            if(len(data[i])>1):
                tmp=tmp+data[i]+' '
    ngram_data.append(tmp)
    """
    for i in range(0,len(data)):
        ngram=gram(data[i])
        #print(ngram)    
        tmp=""
        for k in range(0,len(ngram)):
            for j in range(0,len(ngram[k])):
                tmp=tmp+ngram[j]+" "
            tmp+=data[k]
        ngram_data.append(tmp)
print(ngram_data[0])
"""
print(nltk.pos_tag(ngram_data[3]))
#    for word in data:
        

[('A', 'DT'), ('T', 'NNP'), ('K', 'NNP'), (' ', 'NNP'), ('B', 'NNP'), ('E', 'NNP'), ('5', 'CD'), ('5', 'CD'), ('6', 'CD'), (' ', 'NN')]


In [66]:
from konlpy.corpus import kolaw
from konlpy.tag import *
#from konlpy.tag import Kkma
from konlpy.utils import pprint
#from konlpy.tag import hannanum
#kkma = Kkma()
hanna=Hannanum()
kkma=Kkma()
komo=Komoran()
nouns_list=[]
twitter=Twitter()
#print(tmp_list[0])
#print(test_list[0])
remove_word=['추천', '정품', '고급', '심플', '무료', 
                  '배송', '기능', '일반', '히트', '고성능',
                  '국산', '최고', '퀼리티', '직송', '소량', 
                   '대량', '상담', '코리아', '당일', '도착',
                  '할인', '특가', '가격', '필수', '사용',
                   '가능', '국내','사이즈', '고객', '구매',
                    '대행', '마켓', '유통', '무료', '세트'
                    '사은품', '매장', '미니', '예약', '개입']
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""

    for j in new_list:
        nodap=0
        for remove in remove_word:
            if remove in j:
                nodap=1
        if nodap == 1:
            continue
        #print(j)
        if isNumber(j) is False :
            if(len(j)>1):
                for k in range(0,len(j)):
                    tt+=j[k]
                tt+=' '
    tt+=ngram_data[i]
    tttt=tt[0:len(tt)-1]
    nouns_list.append((x_text_list[i][0],tttt))
#print (nouns_list)


In [68]:

print(nouns_list[0:10])

[('1234621373', '베드 엘레강스 접이 접이식침대 침대 더블 수동 등받이 등받이6단 각도 각도조절 조절 MD 115cm'), ('1543513007', '소형 소형제도판 제도 A3'), ('1546650266', '웨빙 포켓 포켓조끼 조끼 아웃 AGCRIP MK321'), ('1589443530', '텍스 전동접이식침대 동접 이식 침대 전동침대 간병 ATK BE556'), ('1520510486', '플래그 메세지 향초 양초 양초캔 선물 아로마 아로마캔 향초캔 MST'), ('1512822613', '광일 광일체어 체어 스카치 3인 등무 장의자'), ('1525985492', '광일 광일체어 체어 스파 우드 2인 등유 로비 장의자'), ('1326388222', '라움스튜디오 스튜디오 스크류 블랙 블랙봉 커튼 커튼봉 5cm 25mm'), ('1426607155', '모슬 모슬리메모리즈 메모리 산토리 브리즈 아로마 아로마캔 디퓨저 소이 소이캔 소이왁스 왁스'), ('1334473563', '양키 양키캔 악세사리 윅디 심지 심지가위 가위 라이터')]


In [69]:
new_nouns_list=list(map(lambda i : i[1],nouns_list))


print(new_nouns_list[0])

베드 엘레강스 접이 접이식침대 침대 더블 수동 등받이 등받이6단 각도 각도조절 조절 MD 115cm


In [ ]:
'''
noun_train, noun_test, yy_train, yy_test=train_test_split(nouns_list,y_list,test_size=0.08, random_state=42)
print (noun_train[0])
'''

In [70]:
svc_param = np.logspace(-2,0,50)
print(svc_param)
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)
#print(svc_param)
#print(gsvc)

[ 0.01        0.01098541  0.01206793  0.01325711  0.01456348  0.01599859
  0.01757511  0.01930698  0.02120951  0.02329952  0.02559548  0.02811769
  0.03088844  0.03393222  0.03727594  0.04094915  0.04498433  0.04941713
  0.05428675  0.05963623  0.06551286  0.07196857  0.07906043  0.08685114
  0.09540955  0.10481131  0.11513954  0.12648552  0.13894955  0.1526418
  0.16768329  0.184207    0.20235896  0.22229965  0.24420531  0.26826958
  0.29470517  0.32374575  0.35564803  0.39069399  0.42919343  0.47148664
  0.51794747  0.5689866   0.62505519  0.68664885  0.75431201  0.82864277
  0.91029818  1.        ]


In [27]:

gsvc.fit(vectorizer.transform(map(lambda i : i[1],nouns_list)), y_list)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01099,  0.01207,  0.01326,  0.01456,  0.016  ,
        0.01758,  0.01931,  0.02121,  0.0233 ,  0.0256 ,  0.02812,
        0.03089,  0.03393,  0.03728,  0.04095,  0.04498,  0.04942,
        0.05429,  0.05964,  0.06551,  0.07197,  0.07906,  0.08685,
        0.09541...149,
        0.51795,  0.56899,  0.62506,  0.68665,  0.75431,  0.82864,
        0.9103 ,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [28]:
print(gsvc.best_score_, gsvc.best_params_)

0.6876470588235294 {'C': 0.054286754393238594}


In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import datasets


print(gsvc.grid_scores_)

[mean: 0.68718, std: 0.01968, params: {'C': 0.05011872336272722}, mean: 0.68706, std: 0.01995, params: {'C': 0.051348329074375518}, mean: 0.68741, std: 0.01920, params: {'C': 0.052608101759655121}, mean: 0.68741, std: 0.01892, params: {'C': 0.053898781530855235}, mean: 0.68706, std: 0.01916, params: {'C': 0.055221126658075874}, mean: 0.68659, std: 0.01933, params: {'C': 0.056575914014709162}, mean: 0.68600, std: 0.01882, params: {'C': 0.057963939533849672}, mean: 0.68588, std: 0.01898, params: {'C': 0.059386018675902659}, mean: 0.68541, std: 0.01971, params: {'C': 0.060842986907664272}, mean: 0.68518, std: 0.01936, params: {'C': 0.062335700193155576}, mean: 0.68529, std: 0.01936, params: {'C': 0.063865035496498537}, mean: 0.68424, std: 0.01899, params: {'C': 0.06543189129712966}, mean: 0.68471, std: 0.01927, params: {'C': 0.067037188117653509}, mean: 0.68518, std: 0.01912, params: {'C': 0.068681869064646864}, mean: 0.68553, std: 0.01885, params: {'C': 0.070366900382730593}, mean: 0.684

In [29]:
trans=vectorizer.transform(map(lambda i:i[1],noun_train))
gsvc.fit(trans,yy_train)


NameError: name 'noun_train' is not defined

In [ ]:

#print(trans)

In [ ]:
"""
for c in [gsvc.best_params_['C']]:
    clf = LinearSVC(C=c)
    clf.fit(trans, yy_train)
    print (c,clf.score(vectorizer.transform(map(lambda i:i[1],noun_test)), yy_test))
    """

In [71]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))
tmp_list=list(map(lambda i : i[1],eval_x_text_list))
print(tmp_list)


['acchousee 스프레이/기아/오피러스 /2006-2007/Z8/라이트메탈/주문제품(15일예상)', '[엑스포넷주식회사]코일탭_M162.0_18.6mm_1EA 절삭공구 작업공구 공구류 코일탭 탭 공구 빠른배송', '[오가닉맘]펭코남아목도리(MBW7MF02) (인천점)', 'TASCO INFOCUS 골프거리측정기 골프거리 측정 망원경', '(KNF)극동고성능해빙기/고성능전기해빙기/10kw', '(P)인두 플라이어 드라이버등 공구세트 수공구 공구', '[토이트론]실바니안패밀리-유치원학예회세트(3589)', '[누리]사이즈가 크고 튼튼한 뉴테크 좌식 바베큐그릴', '국내산 혀클리너 x 1개(낱개)혀 크리너/입냄새 제거/구강 청결제/여행용/혀간 칫솔 세트/치실/치약/백태', '(추천) 철사밴드_40mm__(통(100EA))/ 호스밴드 호스조임 조임밴드 조임쇠 파이프밴드 철사밴드 철밴드조임', '[핫트랙스] Black 1타스 - [Shachihata] 일본 사찌하타-Artline 유성매직(유성마', 'i1 꽃을든남자 헤나 헤어젤(슈퍼홀드) 500ml/a', '동방애왕뜸기+진영감생명쑥봉(50개)+코끼리라이터', '1300K [투플레이스] 마르니 장스탠드', '김명희 우리애들밥상 고소감자 부침가루', 'HS_905_이지접톱270 절단기 톱날 접이식톱 접톱 절삭', '[ETUDEHOUSE]에뛰드하우스 내 피부를 부탁해 로얄젤리 마스크20ml_1매', 'GOGO!테이스티 우유먼치 (25p)', '1p 지구 스펀지볼(10cm) 미니공 스폰지공 미니볼 스폰지볼 스펀지공 스펀지볼', '[베어파우](BEARPAW) GODETIA NAVY(womens) K460015GD-W', 'H]점멸식 LED 네온/블랙보드 BE127/홍보/메뉴판/광고', 'No350/AK-47에어건/BB탄총/비비탄총', 'S자 스틸 푸쉬업바/요가학원판매용 헬스용품점판매용', '(MST시원한 가야 알로에농장(1.5리터)x12페트 알로에음료 음료수 알로에주스 음료 과일음료', '[GRET](홍콩)홍콩 디즈니랜드

In [72]:
del ngram_data
ngram_data=[]
for ls in tmp_list:
    data = re.findall("[A-Za-z0-9]+", ls)
    tmp=""
    for i in range(0,len(data)):
        if(isNumber(data[i]) is False):
            if(len(data[i])>1):
                tmp=tmp+data[i]+' '
    ngram_data.append(tmp)

In [73]:
test1=[]
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""
    for j in new_list:
        nodap=0
        for remove in remove_word:
            if remove in j:
                nodap=1
        if nodap == 1:
            continue
        if isNumber(j) is False :
            if(len(j)>1):
                for k in range(0,len(j)):
                    tt+=j[k]
                tt+=' '
    tt+=ngram_data[i]
    tttt=tt[0:len(tt)-1]
    test1.append((eval_x_text_list[i][0],tttt))
print(test1[0])


('720988653', '스프레이 기아 오피 오피러스 러스 라이트 라이트메탈 메탈 주문 주문제품 제품 15일예상 예상 acchousee Z8')


In [30]:
pred_list = gsvc.predict(vectorizer.transform(map(lambda i:i[1],test1)))
print(pred_list)

[10  1  2 ...,  6  7  5]


In [31]:

import requests
name='김동현(97)'
nickname='PPAP'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



{'precision': 0.7724458204334366, 'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.'}


In [74]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.dat") as fin:
    for idx,line in enumerate(fin):
#        if idx%100 == 0:
#            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#        print (line)
#        break
        

In [75]:
from scipy import sparse 

In [76]:
'''
img_feature_list = []
for pid, name in noun_train:
    print(pid, name)
    if pid in pid_img_feature_dict:
        img_feature_list.append(pid_img_feature_dict[pid])
#print (len(pid_img_feature_dict[pid]),type(pid_img_feature_dict[pid]))
#         break
    else:
        img_feature_list.append(np.zeros(1000))
#     break
'''

'\nimg_feature_list = []\nfor pid, name in noun_train:\n    print(pid, name)\n    if pid in pid_img_feature_dict:\n        img_feature_list.append(pid_img_feature_dict[pid])\n#print (len(pid_img_feature_dict[pid]),type(pid_img_feature_dict[pid]))\n#         break\n    else:\n        img_feature_list.append(np.zeros(1000))\n#     break\n'

In [77]:
'''
img_feature_test_list = []
for pid, name in noun_test:
    if pid in pid_img_feature_dict:
        img_feature_test_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_test_list.append(np.zeros(1000))
        '''

'\nimg_feature_test_list = []\nfor pid, name in noun_test:\n    if pid in pid_img_feature_dict:\n        img_feature_test_list.append(pid_img_feature_dict[pid])\n    else:\n        img_feature_test_list.append(np.zeros(1000))\n        '

In [78]:
img_feat = []
for pid, name in nouns_list:
    if pid in pid_img_feature_dict:
        img_feat.append(pid_img_feature_dict[pid])
    else:
        img_feat.append(np.zeros(1000))

In [35]:
print(nouns_list[0])
print(noun_test[0])

('1234621373', '베드 엘레강스 접이 접이식침대 침대 더블 수동 등받이 등받이6단 각도 각도조절 조절 MD 115cm')


NameError: name 'noun_test' is not defined

In [79]:

#hi=vectorizer.transform(map(lambda i : i[1],noun_train))
#bye=vectorizer.transform(map(lambda i : i[1],noun_test))
#concat_x_list = sparse.hstack((hi, img_feature_list),format='csr')
#concat_test_x_list = sparse.hstack((bye, img_feature_test_list),format='csr')

hibye=vectorizer.transform(map(lambda i: i[1],nouns_list))

concat_all_list = sparse.hstack((hibye, img_feat),format='csr')


In [105]:
svc_param = np.logspace(-1.24,0,30)
print(svc_param)
gsvc2 = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 5, n_jobs = 4)

[ 0.05754399  0.06349779  0.0700676   0.07731715  0.08531679  0.0941441
  0.10388473  0.11463318  0.12649372  0.13958141  0.15402322  0.16995926
  0.18754412  0.2069484   0.22836034  0.25198768  0.27805962  0.3068291
  0.33857522  0.37360596  0.41226115  0.45491581  0.50198375  0.55392157
  0.61123315  0.67447447  0.74425907  0.82126395  0.90623614  1.        ]


In [106]:

#print(t2)
gsvc2.fit(concat_all_list, y_list)

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.05754,  0.0635 ,  0.07007,  0.07732,  0.08532,  0.09414,
        0.10388,  0.11463,  0.12649,  0.13958,  0.15402,  0.16996,
        0.18754,  0.20695,  0.22836,  0.25199,  0.27806,  0.30683,
        0.33858,  0.37361,  0.41226,  0.45492,  0.50198,  0.55392,
        0.61123,  0.67447,  0.74426,  0.82126,  0.90624,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [83]:
pprint(gsvc2.grid_scores_)
print(gsvc2.best_score_, gsvc2.best_params_)

[mean: 0.68988, std: 0.02016, params: {'C': 0.057543993733715694},
 mean: 0.68976, std: 0.02026, params: {'C': 0.059242254696874941},
 mean: 0.68953, std: 0.02014, params: {'C': 0.060990635405151962},
 mean: 0.68965, std: 0.02037, params: {'C': 0.06279061501216629},
 mean: 0.69000, std: 0.02071, params: {'C': 0.064643716324900624},
 mean: 0.68965, std: 0.02108, params: {'C': 0.066551507092015849},
 mean: 0.68906, std: 0.02114, params: {'C': 0.068515601330187706},
 mean: 0.68965, std: 0.02124, params: {'C': 0.070537660689586404},
 mean: 0.68953, std: 0.02131, params: {'C': 0.072619395859655239},
 mean: 0.68976, std: 0.02154, params: {'C': 0.074762568016376874},
 mean: 0.68976, std: 0.02099, params: {'C': 0.076968990312251684},
 mean: 0.69000, std: 0.02117, params: {'C': 0.079240529410249602},
 mean: 0.69035, std: 0.02110, params: {'C': 0.081579107063031675},
 mean: 0.68976, std: 0.02139, params: {'C': 0.083986701738779118},
 mean: 0.68941, std: 0.02115, params: {'C': 0.08646535029500364

In [ ]:
'''
for c in [0.095]:
    clf2 = LinearSVC(C=c)
    clf2.fit(concat_x_list, yy_train)
    print (c,clf2.score(concat_test_x_list, yy_test))
'''

In [84]:
del pid_img_feature_dict

del pid_img_feature_dict

In [111]:
pid_img_feature_dict = {}
with open("refined_category_dataset.img_feature.eval.dat") as fin:
    for idx,line in enumerate(fin):
#        if idx%100 == 0:
#            print(idx)
        pid, img_feature_str = line.strip().split(" ")
        img_feature = (np.asarray(list(map(lambda i : float(i),img_feature_str.split(",")))))
        pid_img_feature_dict[pid] = img_feature
#         print (line)
#         break
        

In [90]:
test_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        test_x_text_list.append((info['pid'],info['name']))

#pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [112]:
eval_x_text_list = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))

#pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))

In [113]:
img_feature_eval_list = []

In [94]:
for pid, name in test_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))

In [114]:
for pid, name in eval_x_text_list:
    if pid in pid_img_feature_dict:
        img_feature_eval_list.append(pid_img_feature_dict[pid])
    else:
        img_feature_eval_list.append(np.zeros(1000))


In [115]:
print (len(img_feature_eval_list), len(test_x_text_list))

1258 1292


In [96]:
tmp_list=list(map(lambda i : i[1],test_x_text_list))

In [116]:
tmp_list=list(map(lambda i : i[1],eval_x_text_list))
#print(tmp_list)

In [117]:
del ngram_data
ngram_data=[]
for ls in tmp_list:
    data = re.findall("[A-Za-z0-9]+", ls)
    tmp=""
    for i in range(0,len(data)):
        if(isNumber(data[i]) is False):
            if(len(data[i])>1):
                tmp=tmp+data[i]+' '
    ngram_data.append(tmp)

In [118]:
PPAP=[]
for i in range(0,len(tmp_list)):
    new_list=kkma.nouns(tmp_list[i])
    no_num_data=[]
    tt=""
    for j in new_list:
        nodap=0
        for remove in remove_word:
            if remove in j:
                nodap=1
        if nodap == 1:
            continue
        if isNumber(j) is False :
            if(len(j)>1):
                for k in range(0,len(j)):
                    tt+=j[k]
                tt+=' '
    tt+=ngram_data[i]
    PPAP.append((eval_x_text_list[i][0],tt))
print(PPAP)


[('1649273909', '실버 실버카 보행 토모 토모카 레드 레드걸음보조기 걸음 보조기 노인 '), ('1531800813', '프리 프리메라 메라 베이비 쿠션 15g '), ('1596516044', '컬러 GT730 REVENGE D3 1GB LP VGA '), ('1524822010', '앙쥬 욕실 미끄럼 미끄럼방지 미끄럼방지패드 방지 패드 바다 바다이야기 이야기 '), ('1553648730', '핸드 핸드피싱 물방울 물방울추 유동 유동봉돌 봉돌 도래 도래봉돌 해결사 해결사채비 채비 '), ('1303364322', '캐논 바디 렌즈 최신 최신상품 상품 미러 미러리스카메라 리스 카메라 EOS M3 22mm KIT '), ('1647212375', '채혈침 란셋 사혈침 혈침 200입 1박스 박스 28G x1 '), ('1169936535', '현대 현대백화점 백화점 압소 압소바 트상 트상하복 하복 AO7 '), ('1628287970', '트릭 트릭시 고양 고양이하네스세트 이하 네스 세트 블루 블루줄무늬 줄무늬 야광 고양이 고양이목줄 목줄 '), ('1605794098', '창고 슬림 슬림더블지퍼 더블 지퍼 장지갑 지갑 카드 카드지갑 남자 '), ('1173340433', '멸균 멸균우유 우유 6팩 1000mlx6 '), ('1631807051', '살바스 바스 플레이 이태리 수영 수영강습 강습 스노 스노쿨링 쿨링 오리발 PNK '), ('1586496295', '푸르미 애견 애견용 토일렛 '), ('1412268067', '라바 터치 터치펜 옐로 옐로우 터치펜이어폰캡 이어폰 휴대폰 휴대폰악세사리 악세사리 스마트 스마트폰 '), ('1592544098', '위즈위드 위드 데어로 어로 코트 트렌치 트렌치코트 Handmade Joshua Olive Green '), ('1091245840', '화이트 우동기 동기 면기 1P '), ('1563744199', '자이언트 택트 어로 일체형 핸들 핸들바 SLR '), ('1426941582', '삼화 알루미늄

In [119]:
x_feature_list = vectorizer.transform(map(lambda i : i[1],PPAP))
#print(img_feature_eval_list[0])

In [122]:
concat_test_x_list = sparse.hstack((x_feature_list, img_feature_eval_list),format='csr')


In [123]:
pred_list = gsvc2.predict(concat_test_x_list)

In [124]:
print(pred_list[0:10])

[ 7 16  3 13 13 10  7 16  4 14]


In [125]:

import requests
name='김동현(97)'
nickname='PPAP'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())



{'precision': '', 'msg': 'success'}
